In [1]:
import os
import subprocess
from pathlib import Path

"""
Dynamically find the project root (where .git exists) and set it as the current working directory.
"""
project_root = Path(subprocess.check_output(['git', 'rev-parse', '--show-toplevel'], text=True).strip())
os.chdir(project_root)

In [2]:
import pandas as pd
import umap
import plotly.express as px
from sklearn.cluster import DBSCAN
import geopandas as gpd
from src.questions.styleClustering.K_mean_cluster import umap_clustering




In [3]:
BA_US_knn_text = pd.read_csv('data/knnData/BA_US_knn_text.csv')
umap_clustering(BA_US_knn_text)

(50, 824)


/home/belgie/anaconda3/envs/ada/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/belgie/anaconda3/envs/ada/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [3]:
#old PCA clustering

#BA_US_knn_text = pd.read_csv('data/knnData/BA_US_knn_text.csv')

#run_clustering(BA_US_knn_text)

